# スキャルピング銘柄選定分析

## 目的
- 100銘柄×40日分のモックデータで、スキャルピング銘柄が固定化されているか分析
- Entry/Active それぞれのTop 15銘柄の入れ替わり状況を確認
- スコアリング条件の妥当性を検証

In [61]:
import sys
from pathlib import Path

ROOT = Path.cwd().parents[1]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import pandas as pd
import numpy as np
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. データ読み込み

In [62]:
from common_cfg.paths import PARQUET_DIR

TEST_DIR = PARQUET_DIR / "test"
SCORED_PATH = TEST_DIR / "mock_screened_100stocks_scored.parquet"

df = pd.read_parquet(SCORED_PATH)

print(f"Total rows: {len(df):,}")
print(f"Unique stocks: {df['ticker'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of dates: {df['date'].nunique()}")
print(f"\nColumns: {list(df.columns)}")

Total rows: 3,975
Unique stocks: 100
Date range: 2025-08-21 00:00:00 to 2025-10-20 00:00:00
Number of dates: 40

Columns: ['date', 'Open', 'High', 'Low', 'Close', 'Volume', 'ticker', 'prevClose', 'change_pct', 'tr', 'atr14', 'atr14_pct', 'ma5', 'ma25', 'rsi14', 'vol_ma10', 'vol_ratio', 'overall_rating', 'stock_name', 'market', 'sectors', 'series', 'topixnewindexseries', 'entry_filter_passed', 'entry_score', 'entry_rank', 'active_filter_passed', 'active_score', 'active_rank']


## 2. Entry条件分析

In [63]:
# Entry条件通過銘柄のみ抽出
df_entry = df[df['entry_filter_passed']].copy()
df_entry = df_entry.sort_values(['date', 'entry_rank'])

print(f"Entry候補: {len(df_entry):,} rows")
print(f"日付数: {df_entry['date'].nunique()}")
print(f"\n日付ごとの候補数と銘柄:")
entry_daily_count = df_entry.groupby('date').size().sort_index()

# 日付ごとの銘柄名リストを作成
entry_daily_tickers = df_entry.groupby('date').apply(
    lambda x: ', '.join([f"{row['ticker']}({row['stock_name']})" for _, row in x[['ticker', 'stock_name']].iterrows()])
).sort_index()

entry_summary = pd.DataFrame({
    '候補数': entry_daily_count,
    '銘柄': entry_daily_tickers
})

# 表示幅を広げる & 左寄せ
pd.set_option('display.max_colwidth', None)
display(entry_summary.style.set_properties(**{'text-align': 'left'}))

Entry候補: 207 rows
日付数: 39

日付ごとの候補数と銘柄:


,候補数,銘柄
date,,
2025-08-22 00:00:00,8,"1852.T(淺沼組), 197A.T(タウンズ), 2652.T(まんだらけ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-25 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-26 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-27 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト)"
2025-08-28 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-29 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-01 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-02 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-03 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"


### 2.1 Entry Top 15銘柄の推移

In [64]:
# 日付ごとのTop 15（もしくは候補全て）
df_entry_top15 = df_entry[df_entry['entry_rank'] <= 15].copy()

print(f"Entry Top 15相当: {len(df_entry_top15)} rows")
print(f"\n日付ごとのTop 15銘柄数と銘柄:")
entry_top15_daily = df_entry_top15.groupby('date').size().sort_index()

# 日付ごとの銘柄名リストを作成
entry_top15_tickers = df_entry_top15.groupby('date').apply(
    lambda x: ', '.join([f"{row['ticker']}({row['stock_name']})" for _, row in x[['ticker', 'stock_name']].iterrows()])
).sort_index()

entry_top15_summary = pd.DataFrame({
    'Top 15銘柄数': entry_top15_daily,
    '銘柄': entry_top15_tickers
})

# 表示幅を広げる & 左寄せ
pd.set_option('display.max_colwidth', None)
display(entry_top15_summary.style.set_properties(**{'text-align': 'left'}))

Entry Top 15相当: 207 rows

日付ごとのTop 15銘柄数と銘柄:


,Top 15銘柄数,銘柄
date,,
2025-08-22 00:00:00,8,"1852.T(淺沼組), 197A.T(タウンズ), 2652.T(まんだらけ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-25 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-26 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-27 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト)"
2025-08-28 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-08-29 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-01 00:00:00,7,"1852.T(淺沼組), 197A.T(タウンズ), 4176.T(ココナラ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-02 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"
2025-09-03 00:00:00,6,"1852.T(淺沼組), 197A.T(タウンズ), 5852.T(アーレスティ), 6472.T(ＮＴＮ), 6473.T(ジェイテクト), 7184.T(富山第一銀行)"


### 2.2 Entry銘柄の出現頻度

In [65]:
# Top 15に何回登場したか
entry_ticker_freq = df_entry_top15['ticker'].value_counts().sort_values(ascending=False)

print("Entry Top 15に登場した銘柄の出現回数:")
print(f"総銘柄数: {len(entry_ticker_freq)}")
print(f"\n上位20銘柄:")

# 銘柄名を追加
entry_freq_df = entry_ticker_freq.head(20).to_frame('出現回数')
entry_freq_df = entry_freq_df.merge(
    df[['ticker', 'stock_name']].drop_duplicates(),
    left_index=True,
    right_on='ticker',
    how='left'
).set_index('ticker')
display(entry_freq_df[['stock_name', '出現回数']])

# 統計
print(f"\n--- 出現回数統計 ---")
print(f"平均: {entry_ticker_freq.mean():.1f}回")
print(f"中央値: {entry_ticker_freq.median():.1f}回")
print(f"最大: {entry_ticker_freq.max()}回")
print(f"最小: {entry_ticker_freq.min()}回")

Entry Top 15に登場した銘柄の出現回数:
総銘柄数: 11

上位20銘柄:


,stock_name,出現回数
ticker,,
6472.T,ＮＴＮ,39
197A.T,タウンズ,38
1852.T,淺沼組,36
7184.T,富山第一銀行,27
5852.T,アーレスティ,24
6473.T,ジェイテクト,23
3665.T,エニグモ,9
4176.T,ココナラ,6
2652.T,まんだらけ,2



--- 出現回数統計 ---
平均: 18.8回
中央値: 23.0回
最大: 39回
最小: 1回


### 2.3 Entry「常連銘柄」の詳細

In [66]:
# 10回以上登場した「常連銘柄」
entry_regulars = entry_ticker_freq[entry_ticker_freq >= 10].index.tolist()

if len(entry_regulars) > 0:
    print(f"10回以上登場した常連銘柄: {len(entry_regulars)}銘柄")
    
    # 銘柄名を表示
    entry_regulars_with_name = df[df['ticker'].isin(entry_regulars)][['ticker', 'stock_name']].drop_duplicates()
    entry_regulars_with_freq = entry_regulars_with_name.merge(
        entry_ticker_freq.to_frame('出現回数'),
        left_on='ticker',
        right_index=True
    ).sort_values('出現回数', ascending=False)
    display(entry_regulars_with_freq)
    
    # 常連銘柄の詳細データ
    df_entry_regulars = df_entry_top15[df_entry_top15['ticker'].isin(entry_regulars)].copy()
    df_entry_regulars = df_entry_regulars.sort_values(['ticker', 'date'])
    
    print(f"\n常連銘柄の詳細（上位3銘柄のみ）:")
    for ticker in entry_regulars[:3]:
        stock_name = df[df['ticker'] == ticker]['stock_name'].iloc[0]
        print(f"\n--- {ticker} ({stock_name}) ---")
        ticker_data = df_entry_regulars[df_entry_regulars['ticker'] == ticker][[
            'date', 'stock_name', 'Close', 'change_pct', 'Volume', 'vol_ratio', 
            'atr14_pct', 'rsi14', 'overall_rating', 'entry_score', 'entry_rank'
        ]]
        display(ticker_data)
else:
    print("10回以上登場した常連銘柄はありません")

10回以上登場した常連銘柄: 6銘柄


,ticker,stock_name,出現回数
2360,6472.T,ＮＴＮ,39
240,197A.T,タウンズ,38
120,1852.T,淺沼組,36
2840,7184.T,富山第一銀行,27
2240,5852.T,アーレスティ,24
2400,6473.T,ジェイテクト,23



常連銘柄の詳細（上位3銘柄のみ）:

--- 6472.T (ＮＴＮ) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,entry_score,entry_rank
2361,2025-08-22,ＮＴＮ,326.1,2.77,8443100.0,134.46,2.45,100.00,中立,50.0,6.0
2362,2025-08-25,ＮＴＮ,326.3,0.06,4538300.0,79.64,2.38,100.00,中立,50.0,4.0
2363,2025-08-26,ＮＴＮ,322.1,-1.29,3807000.0,72.85,2.35,68.18,中立,63.0,4.0
2364,2025-08-27,ＮＴＮ,322.1,0.00,2934100.0,61.54,2.16,68.18,中立,53.0,5.0
2365,2025-08-28,ＮＴＮ,324.0,0.59,2903900.0,65.15,2.15,72.19,中立,60.0,5.0
2366,2025-08-29,ＮＴＮ,328.9,1.51,6381800.0,134.87,2.41,79.00,中立,65.0,5.0
2367,2025-09-01,ＮＴＮ,330.0,0.33,3558000.0,77.60,2.35,80.09,中立,50.0,5.0
2368,2025-09-02,ＮＴＮ,333.4,1.03,3799900.0,84.48,2.22,82.86,中立,60.0,4.0
2369,2025-09-03,ＮＴＮ,338.7,1.59,8255600.0,169.39,2.27,85.91,中立,70.0,4.0
2370,2025-09-04,ＮＴＮ,342.1,1.00,7184500.0,138.68,2.28,87.35,中立,70.0,3.0



--- 197A.T (タウンズ) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,entry_score,entry_rank
241,2025-08-22,タウンズ,583.0,1.75,692900.0,66.11,3.32,100.00,中立,49.0,2.0
242,2025-08-25,タウンズ,586.0,0.51,534600.0,60.96,3.11,100.00,中立,54.0,2.0
243,2025-08-26,タウンズ,580.0,-1.02,381800.0,50.69,2.86,68.42,中立,59.0,2.0
244,2025-08-27,タウンズ,581.0,0.17,445300.0,64.39,2.64,70.00,中立,49.0,2.0
245,2025-08-28,タウンズ,583.0,0.34,245800.0,39.82,2.37,72.73,中立,50.0,2.0
246,2025-08-29,タウンズ,570.0,-2.23,802500.0,124.66,2.40,45.71,中立,55.0,2.0
247,2025-09-01,タウンズ,578.0,1.40,416400.0,67.67,2.28,55.81,中立,65.0,2.0
248,2025-09-02,タウンズ,575.0,-0.52,367200.0,62.47,2.22,52.17,中立,65.0,2.0
249,2025-09-03,タウンズ,570.0,-0.87,375900.0,66.35,2.08,47.06,中立,65.0,2.0
250,2025-09-04,タウンズ,572.0,0.35,250300.0,55.47,1.92,49.06,中立,53.0,2.0



--- 1852.T (淺沼組) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,entry_score,entry_rank
121,2025-08-22,淺沼組,860.0,0.23,333600.0,120.91,1.05,100.00,中立,54.0,1.0
122,2025-08-25,淺沼組,851.0,-1.05,358400.0,118.13,1.15,18.18,中立,59.0,1.0
123,2025-08-26,淺沼組,843.0,-0.94,155600.0,58.40,1.17,10.53,中立,54.0,1.0
124,2025-08-27,淺沼組,848.0,0.59,294000.0,108.10,1.26,29.17,中立,59.0,1.0
125,2025-08-28,淺沼組,855.0,0.83,184100.0,71.55,1.30,45.16,中立,59.0,1.0
126,2025-08-29,淺沼組,855.0,0.00,139800.0,58.12,1.27,45.16,中立,49.0,1.0
127,2025-09-01,淺沼組,861.0,0.70,321200.0,128.17,1.34,54.05,中立,69.0,1.0
128,2025-09-02,淺沼組,864.0,0.35,255300.0,101.66,1.28,57.50,中立,54.0,1.0
129,2025-09-03,淺沼組,863.0,-0.12,327200.0,126.46,1.31,56.10,中立,59.0,1.0
130,2025-09-04,淺沼組,872.0,1.04,320400.0,119.13,1.34,64.00,中立,62.0,1.0


### 2.4 Entry銘柄の入れ替わり率

In [67]:
# 日付順にソート
dates_sorted = sorted(df_entry_top15['date'].unique())

# 日次の入れ替わり率計算
entry_turnover_rates = []

for i in range(1, len(dates_sorted)):
    prev_date = dates_sorted[i-1]
    curr_date = dates_sorted[i]
    
    prev_tickers = set(df_entry_top15[df_entry_top15['date'] == prev_date]['ticker'])
    curr_tickers = set(df_entry_top15[df_entry_top15['date'] == curr_date]['ticker'])
    
    # 共通銘柄数
    common = len(prev_tickers & curr_tickers)
    # 入れ替わり率 = (新規銘柄数 / 当日銘柄数) * 100
    if len(curr_tickers) > 0:
        turnover_rate = ((len(curr_tickers) - common) / len(curr_tickers)) * 100
    else:
        turnover_rate = 0
    
    entry_turnover_rates.append({
        'prev_date': prev_date,
        'curr_date': curr_date,
        'prev_count': len(prev_tickers),
        'curr_count': len(curr_tickers),
        'common': common,
        'turnover_rate': turnover_rate
    })

df_entry_turnover = pd.DataFrame(entry_turnover_rates)

print("Entry Top 15の日次入れ替わり率:")
display(df_entry_turnover)

print(f"\n--- 入れ替わり率統計 ---")
print(f"平均入れ替わり率: {df_entry_turnover['turnover_rate'].mean():.1f}%")
print(f"中央値: {df_entry_turnover['turnover_rate'].median():.1f}%")
print(f"最大: {df_entry_turnover['turnover_rate'].max():.1f}%")
print(f"最小: {df_entry_turnover['turnover_rate'].min():.1f}%")

Entry Top 15の日次入れ替わり率:


,prev_date,curr_date,prev_count,curr_count,common,turnover_rate
0,2025-08-22,2025-08-25,8,6,6,0.000000
1,2025-08-25,2025-08-26,6,6,5,16.666667
2,2025-08-26,2025-08-27,6,6,5,16.666667
3,2025-08-27,2025-08-28,6,7,6,14.285714
4,2025-08-28,2025-08-29,7,7,7,0.000000
5,2025-08-29,2025-09-01,7,7,7,0.000000
6,2025-09-01,2025-09-02,7,6,6,0.000000
7,2025-09-02,2025-09-03,6,6,6,0.000000
8,2025-09-03,2025-09-04,6,5,5,0.000000
9,2025-09-04,2025-09-05,5,5,4,20.000000



--- 入れ替わり率統計 ---
平均入れ替わり率: 9.0%
中央値: 0.0%
最大: 25.0%
最小: 0.0%


## 3. Active条件分析

In [68]:
# Active条件通過銘柄のみ抽出
df_active = df[df['active_filter_passed']].copy()
df_active = df_active.sort_values(['date', 'active_rank'])

print(f"Active候補: {len(df_active):,} rows")
print(f"日付数: {df_active['date'].nunique()}")
print(f"\n日付ごとの候補数と銘柄:")
active_daily_count = df_active.groupby('date').size().sort_index()

# 日付ごとの銘柄名リストを作成
active_daily_tickers = df_active.groupby('date').apply(
    lambda x: ', '.join([f"{row['ticker']}({row['stock_name']})" for _, row in x[['ticker', 'stock_name']].iterrows()])
).sort_index()

active_summary = pd.DataFrame({
    '候補数': active_daily_count,
    '銘柄': active_daily_tickers
})

# 表示幅を広げる & 左寄せ
pd.set_option('display.max_colwidth', None)
display(active_summary.style.set_properties(**{'text-align': 'left'}))

Active候補: 356 rows
日付数: 39

日付ごとの候補数と銘柄:


,候補数,銘柄
date,,
2025-08-22 00:00:00,6,"2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 4450.T(パワーソリューションズ), 4584.T(キッズウェル・バイオ), 5253.T(カバー)"
2025-08-25 00:00:00,13,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 3665.T(エニグモ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス), 4054.T(日本情報クリエイト), 4176.T(ココナラ), 4584.T(キッズウェル・バイオ), 5253.T(カバー), 6907.T(ジオマテック), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-26 00:00:00,5,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 8115.T(ムーンバット)"
2025-08-27 00:00:00,7,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 4054.T(日本情報クリエイト), 5253.T(カバー), 6200.T(インソース), 8115.T(ムーンバット), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-28 00:00:00,8,"2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 5253.T(カバー), 6907.T(ジオマテック), 8115.T(ムーンバット), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-29 00:00:00,9,"2652.T(まんだらけ), 265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス), 4584.T(キッズウェル・バイオ), 8115.T(ムーンバット), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-09-01 00:00:00,4,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス)"
2025-09-02 00:00:00,7,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3687.T(フィックスターズ), 4176.T(ココナラ), 4584.T(キッズウェル・バイオ), 5253.T(カバー), 6200.T(インソース)"
2025-09-03 00:00:00,9,"265A.T(Ｈｍｃｏｍｍ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 4176.T(ココナラ), 4777.T(ガーラ), 5253.T(カバー), 6907.T(ジオマテック), 7707.T(プレシジョン・システム・サイエンス), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"


### 3.1 Active Top 15銘柄の推移

In [69]:
# 日付ごとのTop 15（もしくは候補全て）
df_active_top15 = df_active[df_active['active_rank'] <= 15].copy()

print(f"Active Top 15相当: {len(df_active_top15)} rows")
print(f"\n日付ごとのTop 15銘柄数と銘柄:")
active_top15_daily = df_active_top15.groupby('date').size().sort_index()

# 日付ごとの銘柄名リストを作成
active_top15_tickers = df_active_top15.groupby('date').apply(
    lambda x: ', '.join([f"{row['ticker']}({row['stock_name']})" for _, row in x[['ticker', 'stock_name']].iterrows()])
).sort_index()

active_top15_summary = pd.DataFrame({
    'Top 15銘柄数': active_top15_daily,
    '銘柄': active_top15_tickers
})

# 表示幅を広げる & 左寄せ
pd.set_option('display.max_colwidth', None)
display(active_top15_summary.style.set_properties(**{'text-align': 'left'}))

Active Top 15相当: 344 rows

日付ごとのTop 15銘柄数と銘柄:


,Top 15銘柄数,銘柄
date,,
2025-08-22 00:00:00,6,"2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 4450.T(パワーソリューションズ), 4584.T(キッズウェル・バイオ), 5253.T(カバー)"
2025-08-25 00:00:00,13,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 3665.T(エニグモ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス), 4054.T(日本情報クリエイト), 4176.T(ココナラ), 4584.T(キッズウェル・バイオ), 5253.T(カバー), 6907.T(ジオマテック), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-26 00:00:00,5,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 8115.T(ムーンバット)"
2025-08-27 00:00:00,7,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 4054.T(日本情報クリエイト), 5253.T(カバー), 6200.T(インソース), 8115.T(ムーンバット), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-28 00:00:00,8,"2743.T(ピクセルカンパニーズ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 5253.T(カバー), 6907.T(ジオマテック), 8115.T(ムーンバット), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-08-29 00:00:00,9,"2652.T(まんだらけ), 265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス), 4584.T(キッズウェル・バイオ), 8115.T(ムーンバット), 8798.T(アドバンスクリエイト), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"
2025-09-01 00:00:00,4,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3964.T(オークネット), 402A.T(アクセルスペースホールディングス)"
2025-09-02 00:00:00,7,"265A.T(Ｈｍｃｏｍｍ), 2743.T(ピクセルカンパニーズ), 3687.T(フィックスターズ), 4176.T(ココナラ), 4584.T(キッズウェル・バイオ), 5253.T(カバー), 6200.T(インソース)"
2025-09-03 00:00:00,9,"265A.T(Ｈｍｃｏｍｍ), 3556.T(リネットジャパングループ), 402A.T(アクセルスペースホールディングス), 4176.T(ココナラ), 4777.T(ガーラ), 5253.T(カバー), 6907.T(ジオマテック), 7707.T(プレシジョン・システム・サイエンス), 9425.T(ＲｅＹｕｕ Ｊａｐａｎ)"


### 3.2 Active銘柄の出現頻度

In [70]:
# Top 15に何回登場したか
active_ticker_freq = df_active_top15['ticker'].value_counts().sort_values(ascending=False)

print("Active Top 15に登場した銘柄の出現回数:")
print(f"総銘柄数: {len(active_ticker_freq)}")
print(f"\n上位20銘柄:")

# 銘柄名を追加
active_freq_df = active_ticker_freq.head(20).to_frame('出現回数')
active_freq_df = active_freq_df.merge(
    df[['ticker', 'stock_name']].drop_duplicates(),
    left_index=True,
    right_on='ticker',
    how='left'
).set_index('ticker')
display(active_freq_df[['stock_name', '出現回数']])

# 統計
print(f"\n--- 出現回数統計 ---")
print(f"平均: {active_ticker_freq.mean():.1f}回")
print(f"中央値: {active_ticker_freq.median():.1f}回")
print(f"最大: {active_ticker_freq.max()}回")
print(f"最小: {active_ticker_freq.min()}回")

Active Top 15に登場した銘柄の出現回数:
総銘柄数: 46

上位20銘柄:


,stock_name,出現回数
ticker,,
2743.T,ピクセルカンパニーズ,30
402A.T,アクセルスペースホールディングス,26
4176.T,ココナラ,24
9425.T,ＲｅＹｕｕ Ｊａｐａｎ,22
265A.T,Ｈｍｃｏｍｍ,19
4584.T,キッズウェル・バイオ,18
5253.T,カバー,18
3556.T,リネットジャパングループ,18
3652.T,ディジタルメディアプロフェッショナル,18



--- 出現回数統計 ---
平均: 7.5回
中央値: 4.0回
最大: 30回
最小: 1回


### 3.3 Active「常連銘柄」の詳細

In [71]:
# 10回以上登場した「常連銘柄」
active_regulars = active_ticker_freq[active_ticker_freq >= 10].index.tolist()

if len(active_regulars) > 0:
    print(f"10回以上登場した常連銘柄: {len(active_regulars)}銘柄")
    
    # 銘柄名を表示
    active_regulars_with_name = df[df['ticker'].isin(active_regulars)][['ticker', 'stock_name']].drop_duplicates()
    active_regulars_with_freq = active_regulars_with_name.merge(
        active_ticker_freq.to_frame('出現回数'),
        left_on='ticker',
        right_index=True
    ).sort_values('出現回数', ascending=False)
    display(active_regulars_with_freq)
    
    # 常連銘柄の詳細データ
    df_active_regulars = df_active_top15[df_active_top15['ticker'].isin(active_regulars)].copy()
    df_active_regulars = df_active_regulars.sort_values(['ticker', 'date'])
    
    print(f"\n常連銘柄の詳細（上位3銘柄のみ）:")
    for ticker in active_regulars[:3]:
        stock_name = df[df['ticker'] == ticker]['stock_name'].iloc[0]
        print(f"\n--- {ticker} ({stock_name}) ---")
        ticker_data = df_active_regulars[df_active_regulars['ticker'] == ticker][[
            'date', 'stock_name', 'Close', 'change_pct', 'Volume', 'vol_ratio', 
            'atr14_pct', 'rsi14', 'overall_rating', 'active_score', 'active_rank'
        ]]
        display(ticker_data)
else:
    print("10回以上登場した常連銘柄はありません")

10回以上登場した常連銘柄: 12銘柄


,ticker,stock_name,出現回数
680,2743.T,ピクセルカンパニーズ,30
1440,402A.T,アクセルスペースホールディングス,26
1600,4176.T,ココナラ,24
3815,9425.T,ＲｅＹｕｕ Ｊａｐａｎ,22
600,265A.T,Ｈｍｃｏｍｍ,19
1000,3556.T,リネットジャパングループ,18
1120,3652.T,ディジタルメディアプロフェッショナル,18
1760,4584.T,キッズウェル・バイオ,18
2080,5253.T,カバー,18
1200,3687.T,フィックスターズ,16



常連銘柄の詳細（上位3銘柄のみ）:

--- 2743.T (ピクセルカンパニーズ) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,active_score,active_rank
681,2025-08-22,ピクセルカンパニーズ,155.0,2.65,5959200.0,106.03,8.47,100.00,売り,69.0,1.0
682,2025-08-25,ピクセルカンパニーズ,205.0,32.26,4792100.0,89.67,8.80,100.00,売り,79.0,2.0
683,2025-08-26,ピクセルカンパニーズ,192.0,-6.34,21405200.0,228.70,13.22,80.60,売り,94.0,2.0
684,2025-08-27,ピクセルカンパニーズ,160.0,-16.67,15191800.0,144.33,17.66,54.55,売り,86.0,2.0
685,2025-08-28,ピクセルカンパニーズ,179.0,11.88,15886500.0,139.12,17.03,61.86,売り,88.0,1.0
686,2025-08-29,ピクセルカンパニーズ,166.0,-7.26,6203900.0,58.12,17.12,55.73,売り,81.0,3.0
687,2025-09-01,ピクセルカンパニーズ,159.0,-4.22,3983100.0,40.49,16.67,52.90,売り,76.0,2.0
688,2025-09-02,ピクセルカンパニーズ,153.0,-3.77,4747200.0,51.20,16.58,50.69,売り,71.0,2.0
690,2025-09-04,ピクセルカンパニーズ,158.0,5.33,3926600.0,46.90,13.61,52.26,売り,81.0,1.0
691,2025-09-05,ピクセルカンパニーズ,154.0,-2.53,1938200.0,24.32,12.79,50.94,売り,66.0,1.0



--- 402A.T (アクセルスペースホールディングス) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,active_score,active_rank
1441,2025-08-22,アクセルスペースホールディングス,853.0,-14.61,21405700.0,112.27,11.93,0.00,売り,84.0,3.0
1442,2025-08-25,アクセルスペースホールディングス,824.0,-3.40,7743100.0,50.63,11.93,0.00,売り,69.0,6.0
1443,2025-08-26,アクセルスペースホールディングス,890.0,8.01,15428700.0,100.67,11.04,27.39,売り,90.0,4.0
1445,2025-08-28,アクセルスペースホールディングス,959.0,9.35,13710500.0,99.94,9.78,44.05,売り,81.0,3.0
1446,2025-08-29,アクセルスペースホールディングス,915.0,-4.59,7840300.0,60.87,9.73,38.95,売り,78.0,5.0
1447,2025-09-01,アクセルスペースホールディングス,852.0,-6.89,5430500.0,45.45,10.15,33.41,売り,83.0,4.0
1449,2025-09-03,アクセルスペースホールディングス,818.0,-4.88,8116400.0,75.60,10.41,31.64,売り,78.0,3.0
1451,2025-09-05,アクセルスペースホールディングス,880.0,7.32,4658800.0,59.65,9.00,39.28,売り,83.0,2.0
1453,2025-09-09,アクセルスペースホールディングス,836.0,-6.07,5235700.0,76.98,9.55,36.83,売り,83.0,6.0
1454,2025-09-10,アクセルスペースホールディングス,986.0,17.94,20901200.0,256.09,9.11,49.15,売り,96.0,4.0



--- 4176.T (ココナラ) ---


,date,stock_name,Close,change_pct,Volume,vol_ratio,atr14_pct,rsi14,overall_rating,active_score,active_rank
1602,2025-08-25,ココナラ,484.0,3.64,515300.0,136.74,2.83,85.00,中立,49.0,8.0
1608,2025-09-02,ココナラ,497.0,3.33,891200.0,193.84,3.33,76.47,中立,70.0,4.0
1609,2025-09-03,ココナラ,518.0,4.23,840400.0,168.82,3.67,83.33,中立,69.0,4.0
1610,2025-09-04,ココナラ,530.0,2.32,848300.0,154.69,3.66,85.71,中立,59.0,3.0
1611,2025-09-05,ココナラ,543.0,2.45,613900.0,105.39,3.74,87.63,中立,54.0,3.0
1613,2025-09-09,ココナラ,510.0,-6.59,1566100.0,215.58,4.80,64.71,中立,93.0,7.0
1614,2025-09-10,ココナラ,497.0,-2.55,593200.0,80.35,4.84,59.06,中立,61.0,6.0
1616,2025-09-12,ココナラ,482.0,-3.60,612400.0,78.40,5.03,49.33,中立,71.0,4.0
1617,2025-09-16,ココナラ,526.0,9.13,981400.0,119.89,5.21,60.82,中立,88.0,6.0
1618,2025-09-17,ココナラ,544.0,3.42,950500.0,115.28,5.32,65.38,中立,78.0,3.0


### 3.4 Active銘柄の入れ替わり率

In [72]:
# 日付順にソート
dates_sorted = sorted(df_active_top15['date'].unique())

# 日次の入れ替わり率計算
active_turnover_rates = []

for i in range(1, len(dates_sorted)):
    prev_date = dates_sorted[i-1]
    curr_date = dates_sorted[i]
    
    prev_tickers = set(df_active_top15[df_active_top15['date'] == prev_date]['ticker'])
    curr_tickers = set(df_active_top15[df_active_top15['date'] == curr_date]['ticker'])
    
    # 共通銘柄数
    common = len(prev_tickers & curr_tickers)
    # 入れ替わり率 = (新規銘柄数 / 当日銘柄数) * 100
    if len(curr_tickers) > 0:
        turnover_rate = ((len(curr_tickers) - common) / len(curr_tickers)) * 100
    else:
        turnover_rate = 0
    
    active_turnover_rates.append({
        'prev_date': prev_date,
        'curr_date': curr_date,
        'prev_count': len(prev_tickers),
        'curr_count': len(curr_tickers),
        'common': common,
        'turnover_rate': turnover_rate
    })

df_active_turnover = pd.DataFrame(active_turnover_rates)

print("Active Top 15の日次入れ替わり率:")
display(df_active_turnover)

print(f"\n--- 入れ替わり率統計 ---")
print(f"平均入れ替わり率: {df_active_turnover['turnover_rate'].mean():.1f}%")
print(f"中央値: {df_active_turnover['turnover_rate'].median():.1f}%")
print(f"最大: {df_active_turnover['turnover_rate'].max():.1f}%")
print(f"最小: {df_active_turnover['turnover_rate'].min():.1f}%")

Active Top 15の日次入れ替わり率:


,prev_date,curr_date,prev_count,curr_count,common,turnover_rate
0,2025-08-22,2025-08-25,6,13,5,61.538462
1,2025-08-25,2025-08-26,13,5,4,20.000000
2,2025-08-26,2025-08-27,5,7,3,57.142857
3,2025-08-27,2025-08-28,7,8,4,50.000000
4,2025-08-28,2025-08-29,8,9,5,44.444444
5,2025-08-29,2025-09-01,9,4,4,0.000000
6,2025-09-01,2025-09-02,4,7,2,71.428571
7,2025-09-02,2025-09-03,7,9,3,66.666667
8,2025-09-03,2025-09-04,9,4,2,50.000000
9,2025-09-04,2025-09-05,4,5,2,60.000000



--- 入れ替わり率統計 ---
平均入れ替わり率: 52.9%
中央値: 55.1%
最大: 100.0%
最小: 0.0%


## 4. スコア分布分析

### 4.1 Entry スコア分布

In [73]:
print("Entry スコアの分布:")
print(df_entry['entry_score'].describe())

print("\nEntry スコアの10分位数:")
for q in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    score = df_entry['entry_score'].quantile(q)
    print(f"{q*100:.0f}%: {score:.1f}")

Entry スコアの分布:
count    207.000000
mean      58.454106
std        8.571038
min       34.000000
25%       53.000000
50%       59.000000
75%       64.500000
max       78.000000
Name: entry_score, dtype: float64

Entry スコアの10分位数:
10%: 48.6
20%: 51.0
30%: 53.0
40%: 56.0
50%: 59.0
60%: 61.0
70%: 63.0
80%: 66.0
90%: 69.0
100%: 78.0


### 4.2 Active スコア分布

In [74]:
print("Active スコアの分布:")
print(df_active['active_score'].describe())

print("\nActive スコアの10分位数:")
for q in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    score = df_active['active_score'].quantile(q)
    print(f"{q*100:.0f}%: {score:.1f}")

Active スコアの分布:
count    356.000000
mean      72.606742
std       13.380767
min       39.000000
25%       64.000000
50%       73.000000
75%       83.000000
max      100.000000
Name: active_score, dtype: float64

Active スコアの10分位数:
10%: 54.0
20%: 61.0
30%: 66.0
40%: 69.0
50%: 73.0
60%: 78.0
70%: 81.0
80%: 84.0
90%: 89.5
100%: 100.0


## 5. 結論と次のアクション

In [75]:
print("="*60)
print("分析結果サマリー")
print("="*60)

print("\n[Entry条件]")
print(f"- 平均候補数/日: {entry_daily_count.mean():.1f}銘柄")
print(f"- Top 15登場銘柄数: {len(entry_ticker_freq)}銘柄")
print(f"- 常連銘柄（10回以上）: {len(entry_regulars)}銘柄")
print(f"- 平均入れ替わり率: {df_entry_turnover['turnover_rate'].mean():.1f}%")

print("\n[Active条件]")
print(f"- 平均候補数/日: {active_daily_count.mean():.1f}銘柄")
print(f"- Top 15登場銘柄数: {len(active_ticker_freq)}銘柄")
print(f"- 常連銘柄（10回以上）: {len(active_regulars)}銘柄")
print(f"- 平均入れ替わり率: {df_active_turnover['turnover_rate'].mean():.1f}%")

print("\n" + "="*60)
print("問題の有無:")
if len(entry_regulars) > 5 or df_entry_turnover['turnover_rate'].mean() < 30:
    print("⚠️  Entry条件: 銘柄の固定化が見られる可能性あり")
else:
    print("✅ Entry条件: 適切に入れ替わっている")

if len(active_regulars) > 5 or df_active_turnover['turnover_rate'].mean() < 30:
    print("⚠️  Active条件: 銘柄の固定化が見られる可能性あり")
else:
    print("✅ Active条件: 適切に入れ替わっている")
print("="*60)

分析結果サマリー

[Entry条件]
- 平均候補数/日: 5.3銘柄
- Top 15登場銘柄数: 11銘柄
- 常連銘柄（10回以上）: 6銘柄
- 平均入れ替わり率: 9.0%

[Active条件]
- 平均候補数/日: 9.1銘柄
- Top 15登場銘柄数: 46銘柄
- 常連銘柄（10回以上）: 12銘柄
- 平均入れ替わり率: 52.9%

問題の有無:
⚠️  Entry条件: 銘柄の固定化が見られる可能性あり
⚠️  Active条件: 銘柄の固定化が見られる可能性あり


In [ ]:
print("=" * 80)
print("チャット貼り付け用結果サマリー")
print("=" * 80)

print("\n### Entry戦略")
print(f"- 平均候補数/日: {entry_daily_count.mean():.1f}銘柄")
print(f"- Top 15登場銘柄数: {len(entry_ticker_freq)}銘柄")
print(f"- 常連銘柄（10回以上）: {len(entry_regulars)}銘柄")
print(f"- 平均入れ替わり率: {df_entry_turnover['turnover_rate'].mean():.1f}%")

print("\n### Entry: 常連銘柄Top 6")
for ticker in entry_regulars[:6]:
    count = entry_ticker_freq[ticker]
    stock_name = df[df['ticker'] == ticker]['stock_name'].iloc[0]
    print(f"{ticker} ({stock_name}): {count}回")

print("\n" + "=" * 80)

print("\n### Active戦略")
print(f"- 平均候補数/日: {active_daily_count.mean():.1f}銘柄")
print(f"- Top 15登場銘柄数: {len(active_ticker_freq)}銘柄")
print(f"- 常連銘柄（10回以上）: {len(active_regulars)}銘柄")
print(f"- 平均入れ替わり率: {df_active_turnover['turnover_rate'].mean():.1f}%")

print("\n### Active: 常連銘柄Top 10")
for ticker in active_regulars[:10]:
    count = active_ticker_freq[ticker]
    stock_name = df[df['ticker'] == ticker]['stock_name'].iloc[0]
    print(f"{ticker} ({stock_name}): {count}回")

print("\n" + "=" * 80)

## 6. 結果コピー用（チャットに貼り付け用）